In [96]:
import numpy as np
import pandas as pd

# READING ALL FILES

In [97]:
# Load order report data from Excel file
order_rept = pd.read_excel("Company ShopX - Order Report.xlsx")

# Load product weight information from Excel file
p_weight = pd.read_excel("Company ShopX - Product Weight.xlsx")

# Load warehouse and customer pin code/zone details from Excel file
pin_zone = pd.read_excel("Company ShopX - Warehouse&Customer Pin Code and Zone details.xlsx")

# Load courier company invoice data from Excel file
invoice = pd.read_excel("Courier Company - Invoice.xlsx")

# Load courier company rate information from Excel file
rates = pd.read_excel("Courier Company - Rates.xlsx")

# OPERATIONS

In [98]:
p_weight.drop_duplicates(inplace=True)

In [99]:
# Merge order report with product weight data
# - Left join keeps all rows from order_rept
# - Matching is done on "Product Code" column
# - This adds weight information to the order details
product_details=pd.merge(left=order_rept,right=p_weight,how="left",on="Product Code")
product_details  # Display the merged dataframe with product details including weights

,Order ID,Product Code,Units Ordered,Product Weight (g)
0,2001827036,8904223818706,1.0,127
1,2001827036,8904223819093,1.0,150
2,2001827036,8904223819109,1.0,100
3,2001827036,8904223818430,1.0,165
4,2001827036,8904223819277,1.0,350
...,...,...,...,...
395,2001806229,8904223818942,1.0,133
396,2001806229,8904223818850,1.0,240
397,2001806226,8904223818850,2.0,240
398,2001806210,8904223816214,1.0,120


In [100]:
# Convert the "Units Ordered" column to numeric values with integer type
# Using downcast="integer" optimizes memory usage by using the smallest integer type possible
product_details["Units Ordered"]=pd.to_numeric(product_details["Units Ordered"],downcast="integer")

In [101]:
# Calculate the total weight in kilograms for each product
# 1. Multiply the number of units ordered by the product weight in grams
# 2. Convert from grams to kilograms by dividing by 1000
# 3. Round the result to 2 decimal places
# 4. Assign the result to a new column called 'total_weight(kg)'
product_details.loc[:,"total_weight(kg)"]=(product_details["Units Ordered"]*product_details["Product Weight (g)"]/1000).round(2)

In [102]:
# Group the product_details dataframe by "Order ID" and calculate the sum of "total_weight(kg)" for each order
# The as_index=False parameter keeps "Order ID" as a column rather than making it an index
# This creates a new dataframe with each order's total weight across all products in that order
order_levels=product_details.groupby("Order ID",as_index=False)[["total_weight(kg)"]].sum()

In [103]:
# Calculate the "weighted slab" by rounding up the total weight to the nearest 0.5 kg
# This creates weight categories or "slabs" for pricing or classification purposes
order_levels.loc[:,"weighted slab"]=np.ceil(order_levels["total_weight(kg)"]/0.5)*0.5

In [104]:
# Calculate additional slabs by:
# 1. Subtracting 0.5 from the "weighted slab" column
# 2. Dividing by 0.5 to convert to slab units
# 3. Converting the result to integer type
# This effectively determines how many additional half-slabs are needed beyond the base level
order_levels.loc[:,"additional slab"]=((order_levels["weighted slab"]-0.5)/0.5).astype(int)

In [105]:
# Merge the order_levels dataframe with the invoice dataframe
# This left join keeps all rows from order_levels and adds matching data from invoice
# The join is performed on the "Order ID" column that exists in both dataframes
order_levels2=pd.merge(left=order_levels,right=invoice,how="left",on="Order ID")
order_levels2  # Display the resulting merged dataframe

,Order ID,total_weight(kg),weighted slab,additional slab,AWB Code (Airway Bill Number),Chargeable Weight,Store House Pincode,Customer Area Code,Delivery Zone,Freight Type,Total Amount (Rs.)
0,2001806210,0.22,0.5,0,1091117221940,2.92,121003,140604,b,Forward charges,174.5
1,2001806226,0.48,0.5,0,1091117222065,0.68,121003,723146,d,Forward charges,90.2
2,2001806229,0.50,0.5,0,1091117222080,0.71,121003,421204,d,Forward charges,90.2
3,2001806232,1.30,1.5,2,1091117222124,1.30,121003,507101,d,Forward charges,135.0
4,2001806233,0.25,0.5,0,1091117222135,0.78,121003,263139,b,Forward charges,61.3
...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,0.48,0.5,0,1091121183730,0.50,121003,342008,d,Forward charges,45.4
120,2001822466,1.38,1.5,2,1091121305541,1.10,121003,342301,d,Forward charges,135.0
121,2001823564,0.66,1.0,1,1091121666133,0.70,121003,492001,d,Forward and RTO charges,172.8
122,2001825261,1.56,2.0,3,1091121981575,1.60,121003,517128,d,Forward and RTO charges,345.0


In [106]:
pin_zone=pin_zone.drop_duplicates()
pin_zone

,Store House Pincode,Customer Area Code,Delivery Zone
0,121003,507101,d
1,121003,486886,d
2,121003,532484,d
3,121003,143001,b
4,121003,515591,d
...,...,...,...
114,121003,324008,b
115,121003,302020,b
119,121003,325207,b
120,121003,303702,b


In [107]:
# Merge order_levels2 dataframe with pin_zone dataframe
# Left join keeps all records from order_levels2 and matching records from pin_zone
# Join is performed on "Store House Pincode" and "Customer Area Code" columns
# Suffix "_courier" is added to duplicate column names from order_levels2
# Suffix "_shopX" is added to duplicate column names from pin_zone
order_levels3=pd.merge(left=order_levels2,right=pin_zone,
         how="left",
         on=["Store House Pincode","Customer Area Code"],
         suffixes=("_courier","_shopX")
        )

In [108]:
# Calculate the courier weight slab by:
# 1. Dividing the chargeable weight by 0.5 to get number of half-kg units
# 2. Rounding up to the nearest integer using np.ceil()
# 3. Multiplying back by 0.5 to get the weight in kg
# This creates weight slabs in 0.5kg increments (0.5kg, 1kg, 1.5kg, etc.)
order_levels3["Courier Weight Slab"]=np.ceil(order_levels3["Chargeable Weight"]/0.5)*0.5

In [109]:
# Lambda function to retrieve the fixed forward rate for a given zone z
lambda z:rates.loc[0,f"fwd_{z}_fixed"]
# Lambda function to retrieve the additional forward rate for a given zone z
lambda z:rates.loc[0,f"fwd_{z}_additional"]
# Lambda function to retrieve the fixed return-to-origin rate for a given zone z
lambda z:rates.loc[0,f"rto_{z}_fixed"]
# Lambda function to retrieve the additional return-to-origin rate for a given zone z
lambda z:rates.loc[0,f"rto_{z}_additional"]

<function __main__.<lambda>(z)>

In [110]:
# Map forward fixed rates based on delivery zone from rates dataframe
order_levels3["fwd_fixed"]=order_levels3["Delivery Zone_shopX"].map(lambda z:rates.loc[0,f"fwd_{z}_fixed"])

# Map forward additional rates based on delivery zone from rates dataframe
order_levels3["fwd_additional"]=order_levels3["Delivery Zone_shopX"].map(lambda z:rates.loc[0,f"fwd_{z}_additional"])

# Map return-to-origin fixed rates based on delivery zone from rates dataframe
order_levels3["rto_fixed"]=order_levels3["Delivery Zone_shopX"].map(lambda z:rates.loc[0,f"rto_{z}_fixed"])

# Map return-to-origin additional rates based on delivery zone from rates dataframe
order_levels3["rto_additional"]=order_levels3["Delivery Zone_shopX"].map(lambda z:rates.loc[0,f"rto_{z}_additional"])

In [111]:
# Calculate expected forward charges by adding fixed charges and additional charges
# Formula: expected_fwd_charges = fwd_fixed + (fwd_additional * additional slab)
order_levels3["expected_fwd_charges"]=order_levels3["fwd_fixed"]+order_levels3["fwd_additional"]*order_levels3["additional slab"]

In [112]:
# Calculate expected RTO (Return to Origin) charges based on freight type
# If freight type is not "Forward charges", calculate RTO charges using fixed cost plus additional cost per slab
# Otherwise, set RTO charges to 0
order_levels3["expected_rto_charges"]=np.where(order_levels3["Freight Type"]!="Forward charges",
         order_levels3["rto_fixed"]+
         order_levels3["rto_additional"]*order_levels3["additional slab"],
         0)

In [113]:
# Calculate total expected charges for shopX by adding forward charges and RTO (Return to Origin) charges
order_levels3["expected_charges_shopX"] = order_levels3["expected_fwd_charges"] + order_levels3["expected_rto_charges"]

In [114]:
# Calculate the difference between actual total amount and expected charges from shopX
# Round the result to 2 decimal places for better readability
order_levels3["difference"] = (order_levels3["Total Amount (Rs.)"] - order_levels3["expected_charges_shopX"]).round(2)

In [115]:
# Determine the charging status based on the 'difference' column:
# - If difference is 0: "Correctly charged"
# - If difference is positive: "Overcharged"
# - If difference is negative: "Undercharged"
order_levels3["status"]=np.where(order_levels3["difference"]==0,
                                 "Correctly charged",np.where(order_levels3["difference"]>0,
                                                              "Overcharged","Undercharged"
                                                             )
                                )

In [116]:
# Rename columns to more descriptive names for better readability
# - Convert AWB Code to simpler "AWB Number"
# - Clarify weight columns by specifying the source (ShopX vs Courier Company)
# - Standardize naming convention for weight-related columns
order_levels3.rename(columns={"AWB Code (Airway Bill Number) ":"AWB Number",
                              "total_weight(kg)":"Total weight as per ShopX (KG)",
                              "weighted slab":"Weight slab as per ShopX (KG)",
                              "Chargeable Weight":"Total weight as per Courier Company (KG)",
                              "Courier Weight Slab":"Weight slab charged by Courier Company (KG)"},
                     inplace=True
                    )

In [117]:
# Create a new DataFrame 'Calculations' with selected columns from 'order_levels3'
# This DataFrame contains order details, weight information from both ShopX and Courier Company,
# delivery zone information, charge calculations, and status information
Calculations=order_levels3[["Order ID","AWB Number",
                            "Total weight as per ShopX (KG)","Weight slab as per ShopX (KG)",
                            "Total weight as per Courier Company (KG)",
                            "Weight slab charged by Courier Company (KG)",
                            "Delivery Zone_shopX","Delivery Zone_courier",
                            "expected_charges_shopX","Total Amount (Rs.)","difference","status"]]

In [118]:
Calculations

,Order ID,AWB Number,Total weight as per ShopX (KG),Weight slab as per ShopX (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone_shopX,Delivery Zone_courier,expected_charges_shopX,Total Amount (Rs.),difference,status
0,2001806210,1091117221940,0.22,0.5,2.92,3.0,b,b,33.0,174.5,141.5,Overcharged
1,2001806226,1091117222065,0.48,0.5,0.68,1.0,d,d,45.4,90.2,44.8,Overcharged
2,2001806229,1091117222080,0.50,0.5,0.71,1.0,d,d,45.4,90.2,44.8,Overcharged
3,2001806232,1091117222124,1.30,1.5,1.30,1.5,d,d,135.0,135.0,0.0,Correctly charged
4,2001806233,1091117222135,0.25,0.5,0.78,1.0,b,b,33.0,61.3,28.3,Overcharged
...,...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,1091121183730,0.48,0.5,0.50,0.5,b,d,33.0,45.4,12.4,Overcharged
120,2001822466,1091121305541,1.38,1.5,1.10,1.5,b,d,89.6,135.0,45.4,Overcharged
121,2001823564,1091121666133,0.66,1.0,0.70,1.0,d,d,176.3,172.8,-3.5,Undercharged
122,2001825261,1091121981575,1.56,2.0,1.60,2.0,d,d,355.5,345.0,-10.5,Undercharged


In [119]:
# Group data by 'status' and calculate:
# - count: Number of orders in each status
# - Amount: Sum of 'difference' values for each status
# - Billed_Amount: Sum of 'expected_charges_shopX' for each status
summary=Calculations.groupby("status",as_index=False).agg(count=("Order ID","count"),
                                           Amount=("difference","sum"),
                                           Billed_Amount=("expected_charges_shopX","sum"))

In [120]:
summary

,status,count,Amount,Billed_Amount
0,Correctly charged,23,0.0,2006.7
1,Overcharged,79,4483.2,3986.6
2,Undercharged,22,-530.3,3702.0


# TRANSFORMING INSIGHTS IN EXCEL SHEET

In [121]:
# Create a list of dataframes to export
dfs=[summary,Calculations]
# Create a list of sheet names for the Excel file
sheets=["summary","Calculations"]
# Create an Excel writer to save the dataframes to a file called "Logistics_project.xlsx"
with pd.ExcelWriter("Logistics_project.xlsx") as writer:
    # Loop through each dataframe and sheet name pair
    for df,name in zip(dfs,sheets):
        # Write each dataframe to the Excel file in its corresponding sheet
        # The index=False parameter prevents row indices from being included
        df.to_excel(writer,sheet_name=name,index=False)

# ZIPPING THE EXCEL FILE

In [122]:
# Import the zipfile module to work with ZIP archives
import zipfile
# Create a new ZIP file named "Logistics_project.zip" in write mode
with zipfile.ZipFile("Logistics_project.zip",mode="w") as z:
    # Add the Excel file to the ZIP archive
    z.write("Logistics_project.xlsx")
    # Add the Jupyter notebook file to the ZIP archive
    z.write("Logistics_project.ipynb")